In [10]:
%pip install fuzzywuzzy

In [11]:
import numpy as np
import pandas as pd

import random
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [12]:
home_url = 'https://ru-paronym.ru'
data_filename = "paronym.csv"

In [13]:
paronyms = []

# взятие паронимов с открытого ресурса
ru_alphabet = [chr(i) for i in range(ord('А'), ord('А') + 32)]
no_words = ['Ё', 'Й', 'Щ', 'Ъ', 'Ы', 'Ь']

# Создаем множество всех слов из датасета паронимов
all_words = set()

for letter in ru_alphabet:
    if letter in no_words:
        continue
    url = home_url + '/' + letter
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    arr = soup.find_all('a', href=True)[2:-27]  # Откидываем первые 2 и последние 27 элемента, которые есть на каждой странице
    for elem in arr:
        webpage = home_url + elem['href']
        words = (elem.string).split(' — ')
        paronyms.append([words[0], words[1], 1])  # пары слов - паронимы
        paronyms.append([words[1], words[0], 1])  # пара в обратном порядке
        all_words.update((words[0], words[1])) # Создаем множество всех слов из датасета паронимов

In [14]:
# база данных с кортежами с меткой (1) - паронимы
print(paronyms[:5])

[['абонемент', 'абонент', 1], ['абонент', 'абонемент', 1], ['абстрактность', 'абстракция', 1], ['абстракция', 'абстрактность', 1], ['авантюрин', 'авантюрист', 1]]


In [16]:
data = pd.DataFrame(paronyms, columns=['word1', 'word2', 'label'])
data.to_csv("only_paronyms.csv")

In [17]:
# создание пар слов не паронимов путем перемешивания уже имеющихся слов
def generate_non_paronyms(paronyms, seed = None, ratio = 1):

    non_paronyms = []

    if seed:
      random.seed(seed)

    # Случайные пары из множества всех слов
    for _ in range(int(len(paronyms) * ratio )):
        word1, word2 = random.choices(list(all_words), k=2)
        if (word1 + ' ' + word2, 1) not in paronyms:
          non_paronyms.append([word1, word2, 0])  # Метка 0, так как это непаронимы

    return non_paronyms

non_paronyms = generate_non_paronyms(paronyms, 42)

In [18]:
dataset_par = paronyms + non_paronyms
random.shuffle(dataset_par) # перемешивание датасета на всякий случай
print(dataset_par[:5])

[['клетчатый', 'грамматический', 0], ['целый', 'цельный', 1], ['заводской', 'вскрытие', 0], ['нафталин', 'чародейный', 0], ['сыскать', 'снискать', 1]]


In [19]:

data = pd.DataFrame(dataset_par, columns=['word1', 'word2', 'label'])
data.to_csv(data_filename)
